## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach, Simon Schäffler

**contact:** ingmar.nitze@awi.de

**version date:** 2022-02-08

**repository and other tools** https://github.com/awi-response/MACS_tools

### Imports 

In [ ]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path
from joblib import delayed, Parallel, wrap_non_picklable_objects
import numpy as np
from pathlib import Path
from processing_utils import *

In [ ]:
pwd = Path(os.getcwd())

### Functions 

### Settings 
* prefer full/absolute paths
* Create processing template automatically

#### Variable Settings 

In [ ]:
# Set project directory here, where you want to process your data
PROJECT_DIR = r''
#PROJECT_DIR = r'D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_non_cropped' # SET Project output

# Set raw data dir here for the speicific image acquisition project
path_infiles = r''
#path_infiles = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190727-235440_15L_Ketik_fire_flight_plan_v3'

# determine which sensors to include in processing (possible options: 'left', 'right', 'nir')
sensors = ['left', 'right', 'nir']

# Set CROP CORNER if 
CROP_CORNER = 0 # SET to 1 if you want to crop corners (set to NoData)
DISK_SIZE = 5200 # Cropping diameter, the larger the fewer no data

#### Fixed Settings
* These Settings can be kept

In [ ]:
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'
EXIF_PATH = Path(CODE_DIR / Path(r'exiftool\exiftool.exe'))
mipps_script_dir = Path('mipps_scripts')

In [ ]:
mipps_script_nir = '33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
mipps_script_right = '33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
mipps_script_left = '33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

#mipps_script_nir = r'mipps_scripts\33552_all_taps_2018-09-26_12-58-15_modelbased_x4.mipps'
#mipps_script_right = r'mipps_scripts\33576_all_taps_2018-09-26_13-13-43_modelbased_x4.mipps'
#mipps_script_left = r'mipps_scripts\33577_all_taps_2018-09-26_13-21-24_modelbased_x4.mipps'

mipps_script_nir = pwd / mipps_script_dir / mipps_script_nir
mipps_script_right = pwd / mipps_script_dir / mipps_script_right
mipps_script_left = pwd / mipps_script_dir / mipps_script_left

In [ ]:
DATA_DIR = Path(PROJECT_DIR) / '01_rawdata' / 'tif'
OUTDIR = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

In [ ]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [ ]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [ ]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [ ]:
#df_final = prepare_df_for_mipps(path_footprints, path_infiles)
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

#### Workaround to deal with spaces in path" 

In [ ]:
df_final['full_path'] = df_final.apply(lambda x: f'"{x.full_path}"', axis=1)

In [ ]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

#### Run Process 

In [ ]:
os.chdir(MIPPS_DIR)

In [ ]:
max_roll = 3 # Select maximum roll angle to avoid image issues - SET in main settings part?
chunksize = 20 # this is a mipps-script thing

In [ ]:
# this is relevant for NIR only
if 'nir' in sensors:
    looking = 'center'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_nir = df_final[q]
    print(len(df_nir))
    for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
        os.system(s)
        #print(s)

In [ ]:
# this is RGB
if 'right' in sensors:
    looking = 'right'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_right = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
        os.system(s)

In [ ]:
if 'left' in sensors:
    looking = 'left'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_left = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
        os.system(s)

#### Crop Corners of images 

* joblib will crash
* in other notebook it did not crash

In [ ]:
if CROP_CORNER:
    #mask = make_mask((3232, 4864), disksize=DISK_SIZE)
    for sensor in sensors[:]:
        mask = make_mask((3232, 4864), disksize=DISK_SIZE)
        images = list(OUTDIR[sensor].glob('*'))
        if sensor != 'nir':
            mask = np.r_[[mask]*3]
        %time _ = [mask_and_tag(image, mask, tag=None) for image in tqdm.tqdm_notebook(images)]
        #%time _ = Parallel(n_jobs=4)(delayed(mask_and_tag)(image, mask, tag=None) for image in tqdm.tqdm_notebook(images))

#### Write exif information into all images 

In [ ]:
for sensor in tqdm.tqdm_notebook(sensors):
    print(sensor)
    %time write_exif(OUTDIR[sensor], tag[sensor], EXIF_PATH)

#### Nav

In [ ]:
navfile = list(Path(path_infiles).glob('*nav.txt'))[0]

In [ ]:
shutil.copy(navfile, OUTDIR['nir'].parent / 'nav.txt')

In [ ]:
os.chdir(OUTDIR['nir'].parent)
os.system('python pix4dnav.py')